<nav style="text-align:center;">
    <img style="float:right" src="./Data/Images/Report/0000.Springboard.jpg" width="70" height="70" />
    <span style="float:center;text-align:center">
        <p style="text-align:center;margin:0;font-size:12px;color:#c1121f"> <b> Data Science = Solving Problems = Happiness </b> </p>
        <p style="text-align:center;font-size:40px;margin:0"> <b> Bike Share USA</b> </p>
        <p style="text-align:center;margin:0"> Denzel S. Williams </p>
        <p style="text-align:center;margin:0"> <i> Springboard Data Science Track '20 </i> </p>
    </span>
</nav>


## 0. Introduction

Every major human advancement improved how people, things, or ideas moved from one point to another. Ancient innovations such as agriculture, providing us with a surplus of food, enabled us to stop moving and build civilizations. Present day innovations such as the Internet has taken movement to all new levels. Leveraging the internet, you can order a package and have it delivered by day end or you can send software money, permissionlessly, from one end of the world to another. More so, the internet enables the global movement of information at near instant speeds. 

When it comes to the physical transportation of people, intra-planet space travel and self-driving cars are the talk of the town. However, the greatest macro transportation revolution is happening on a micro level.  Micromobility refers to a range of small, lightweight vehicles operating at speeds typically below 15-mph and driven by users personally. Micromobility devices include, bicycles, e-bikes, electric scooters and skateboards, shared bicycles, and electric pedal assisted bicycles <a href="https://en.wikipedia.org/wiki/Micromobility" target="_blank"> [1]</a>. 

<div style="clear: right;">
   <p style="float: right;"> 
       <figure style="float:right;text-align:center">
           <div>
               <img src="./Data/Images/Report/0001.Shared-Micromobility-Graph.png" width="600" height="600" />
           </div>
           <div>
               <img src="./Data/Images/Report/0002.Shared-Micromobility-Map.png" width="600" height="600" />
           </div>
           <figcaption> Figure 1. NACTO 2019 Shared Micromobility Report Graphs </figcaption>
       </figure>
   </p>
     <p>According to the 2019 Shared Micromobility Snapshot, published by the National Association of City Transportation Officials (NACTO), the number of trips taken on shared bicycles, e-bikes, and scooters was 136 million; up 60% from 2018’s 84 million and 288% from 2017’s 35 million <a href="https://nacto.org/shared-micromobility-2019/"> [2]</a>.  Although there is not any official data about transportation mode shifting, their survey data suggests that Micromobility might be replacing car trips. The United States (US) has about 19,495 incorporated cities, towns, and villages and of those, 310 are considered at least medium cities with populations of 100,000 or more <a href="https://worldpopulationreview.com/us-city-rankings/how-many-cities-are-in-the-us"> [3]</a>. Looking at the NACTO map there are only about 130 cities that have micromobility services. Imagine if there were micromobility services in all 310 of those cities. Better yet, imagine every part of the United States having micromobility services and instead of a sparse map the shared micromobility map resembled a Verizon 4G LTE Coverage map. 
         <p style="text-align:center;"> <b>The goal of this project is to expand the station-based bike sharing sector of a State's micromobility services. The idea is to use both zip code and bike sharing data of areas that have stations to build a model that can predict the number of stations that should be built in no-station zip codes. The question that this project is looking to answer is: How many bike sharing stations should be built in the zip codes that already have bike stations within the States of: CA, MA, D.C., MD, VA, NJ, NY, and IL?</b></p> 
    
<h2> I. The Data Engineering </h2>

The data used to complete the project can be broken into four major groups. The first two groups were fundamental to completing the project and the other half were required for the Exploratory Data Analytics (EDA):

<b> Bike Share Trip Datasets </b> <br>
The subset of zip codes that have bike stations are derived from the five largest bike sharing services in the US: Bay Wheels, Blue Bike, Capital Bikeshare, Citi Bike, and Divvy Bike. Each company hosts their trip data on S3 buckets for public use. 
These datasets hold key information about each trip that was taken by their customers. The rows of the datasets represent a single trip; the columns are the properties of the trip such as the starting station and the time when the trip ended. Since the trip data includes the start and end stations, the station data used to make the predictions was derived from these datasets.

<b> Zip Code Datasets </b> <br>
All the zip codes of the US along with the properties of the zip code are included in this group. Properties such as the total population, core based statistical area classification, and water area are included. Two supplementary files were used to help fill missing values within the main zip code file. 
    
<b> Geospatial Datasets </b> <br>
New York City (NYC) and San Francisco has geospatial boundaries of their segmented neighborhoods. The datasets in this group contain those geospatial multi-polygons. Additionally, the geospatial locations of the MTA Subway Entrances was gathered for particular analytics section <i>(2.2.E Would You Have Bike Access?)</i>.
    
<b> Neighborhood Profile Datasets </b> <br>
The datasets in this group have the demographics of those segmented neighborhoods. These demographics, when combined with the geospatial data were used to do two custom analyses in the EDA portion of the project. The analysis used both the station location point geometries and the Voronoi polygons of the station locations. 
    </p>
</div>

<p style="text-align:center;"> <b> 1.1 ETL & The Database</b> </p>

All together there were ten different datasets which summed to 68 GB of data across 350+ files. To work with this data, the best course of action was to build a database. Leveraging the Amazon Web Services (AWS) Cloud a RDS Database running PostgreSQL 12.5 was created on a db.t3.micro instance.  

With the blank database created, before doing anything, it was important to think about how the data was going to be used for analytics; to determine how it should be feed into the database. With that idea in mind an Entity Relationship Diagram (ERD) was created to structure the database and guide the transformation portion of the upcoming Extract Transform and Load (ETL) jobs. 

<figure style="text-align:center">
    <img src="./Data/Images/Report/0003.ERD-Final.png" style="max-width:65%" />
    <figcaption style="text-align:center"> Figure 2. Entity Relationship Diagram of the Database </figcaption>
</figure>

<p style="text-align:right"> <a href="#Dataset-Usage"> Link to A2. Dataset Usage Table </a> </p>

<a id="Jump-To-Cleaning"> </a>

<p style="text-align:right"> <b> 1.1.A Extract</b> </p>
    
The first part of the project was a series of massive ETL jobs. The data had to be extracted from websites, S3 buckets, zip folders, csv files, pdf files, excel files, and geo spatial files. As seen with using RDS, one thing the project reached for was to leverage the basic services of AWS. Therefore all the data was extracted and <i> indirectly </i> uploaded to a personal S3 bucket. 
<p style="text-align:center"> <i> The data was extracted to a local temporary folder and then bulk uploaded to the S3 bucket instead of being uploaded directly to the bucket after extraction. </i> </p>

Only the bike share trip data and the neighborhood profile data needed to be extracted via code. The other two groups were a simple download. For each individual bike service, a custom set of functions and loops were created to request all the relevant files from that company's S3 bucket. The requests returned a zipfile which was unzipped; where only the relevant files were filtered out, extracted and saved. 
    
The NYC neighborhood data is hosted at the <a href="https://furmancenter.org/neighborhoods"> Furman Center </a>. On the website, each neighborhood has its own page that a user can get to via a dropdown menu. On each neighborhood page, the packaged data can be downloaded as an excel file. Using the BeautifulSoup package, the dropdown menu was scraped to get the codes which could then be used to construct a series of direct requests to all the packaged data across the site.
    
When it comes to the San Francisco data it had to be pulled out of a PDF file. The file was a report that originated from the <a href="https://default.sfplanning.org/publications_reports/SF_NGBD_SocioEconomic_Profiles/2012-2016_ACS_Profile_Neighborhoods_Final.pdf"> SF Planning Department</a> and the data in the file was structured exactly the same for each neighborhood. Using the PyPDF2 package, all of the relevant pages of the report were iteared through, extracting out the lines of data needed along the way.

<p style="text-align:right"> <b> 1.1.B Transform</b> </p>

After the extraction, three main technologies were used in the transformation phase: Python and the Pandas package, PostgreSQL, and the Google Cloud Platform (GCP) Geocoding API. 
    
The trip dataset files were going to be sent to their appropriate staging table which meant their formats had to align with the staging table design. This required replacing missing values, reordering columns, inserting columns, changing data types, and doing string replacements for every trip file for every service. To handle this, like the extraction phase, custom functions were made to transform all the trip files within a bike service. Once transformed loading the data into the database took only a handful of lines of code. 
       
<figure style="text-align:center">
    <figcaption style="text-align:center;margin: 10px 0;"> Table 1. A sample entry from a table in the staging schema. </figcaption>
    <img src="./Data/Images/Report/0004.Staging-Sample.jpg" style="max-width:100%" />
</figure>

Embedded within every individual trip is the station information for both the starting and ending station of the trip. With all the trip data inside their staging tables a simple <code>DISTINCT</code> clause embedded within a query returned a dataframe that contained all the unique stations and the information associated with them. 

When pulled down from the database, most stations had latitude and longitude data that was used to append a point geometry to the returned dataframe. However, the core piece of data needed to complete the project, the zip code, didn’t exist. To get the zip codes the geographic coordinates were sent to the GCP Geocoding API and the zip code was extracted. By 2020 most stations had coordinates attached to them. If the coordinates were not available, the name of the station was sent to the GCP Geocoding API combined with Region Filtering and the zip code was extracted. If both things failed the zip code was manually found through different search methods. After being located the zip codes were then appended to the dataframe. With the zip codes in place the station data aligned with the database design and was loaded into the corresponding station table. 

<figure style="text-align:center">
    <figcaption style="text-align:center;margin: 10px 0;"> Table 2. A sample entry from a table in the station schema. </figcaption>
    <img src="./Data/Images/Report/0005.Station-Sample.jpg" style="max-width:60%" />
</figure>

Using the data in the staging and stations schema, the trip table was created within the database using a query to select the desired columns. The query also manufactured 4 artificial columns:

<ol>
    <li> Subtracting the starting time from the ending time of the trips returned the <b> duration </b> in minutes.
    <li> Using the station point geometries the <b> distance </b> between the starting station and the ending station was calculated in miles.
    <li> If both the duration and the distance column weren’t null, then the <b> speed </b> of the trip was calculated in mph.
    <li> The name of the <b> service </b> for each trip was a constant value for every trip within a trip table. 
</ol>

<figure style="text-align:center">
    <figcaption style="text-align:center;margin: 10px 0;"> Table 3. A sample entry from a table in the trips schema. </figcaption>
    <img src="./Data/Images/Report/0006.Trip-Sample.jpg" style="max-width:100%" />
</figure>

The NYC Neighborhood data downloaded from the Furman Center was split across two tables. The profile table contained the actual data and the lookup table was used to look up the name and the descriptions of the columns in the profile table.
The properties of the NYC Neighborhoods are grouped into different categories, such as Demographics and Housing Market/Conditions. In the lookup table each property is given an alias based on the group that it is in and its position in the group. For example, the percentage of people born in New York State is the first property in the demographics group, so it is aliased DEM1. Additionally, the full name and description will get their own columns.
    
<figure style="text-align:center">
    <figcaption style="text-align:center;margin: 10px 0;"> Table 4. A sample of entries from the nyc_lookup table in the neighborhoods schema. </figcaption>
    <img src="./Data/Images/Report/0007.Alias-Sample.jpg" style="max-width:100%" />
</figure>    
Each NYC Neighborhood excel file has, as rows, the properties of the neighborhood; the columns of the files are different years of recorded data. In the project only the most recent 2018 data was used. Each of the 59 files had to be individually pivoted into a single row. The single rows were then glued together to create one dataframe that was sent to the database.

<figure style="text-align:center">
    <figcaption style="text-align:center;margin: 10px 0;"> Table 5. A sample of entries from the nyc_profile table in the neighborhoods schema.</figcaption>
    <img src="./Data/Images/Report/0008.Profile-Sample.jpg" style="max-width:70%" />
</figure>
    
<i> The zip code data did not require that many transformations before sending it to the database. The only things that were done were changing column names and filling missing values with a place holder.</i>  

<p style="text-align:center;"> <b> 1.2 Cleaning & Updating</b> </p>

The stations were derived from the entire history of trips. The problem with this is that the ecosystem of stations in a bike sharing service is always changing. The services add new stations, remove stations, and occasionally move stations to nearby locations. It is important that the current state of the ecosystem be known when making predictions about zip codes. The model shouldn’t count a retired station in a zip code's station count. Using the trip table, two new columns were added to the tables in the station schema. The birth column recorded the first time a station appeared in a trip and the death column recorded the last time a station appeared in a trip. From there, stations that had a death date within December 2020 were considered alive and their death dates were set to NULL. The birth and death columns unlocked the ability to look at the state of the ecosystems at any point in time. 

<figure style="text-align:center">
    <figcaption style="text-align:center;margin: 10px 0;"> Table 6. A sample entry from a table in the stations schema after the birth certificate update.</figcaption>
    <img src="./Data/Images/Report/0008.Station-Birth-Sample.jpg" style="max-width:100%" />
</figure>

<p style="text-align:right"> <b> 1.2.A Trip Table Cleaning - Basic Cleaning</b> </p>

The first set of cleaning regarding the trip table was handling outliers in the speed column. Trips that had speed values too high had to be handled different from speed values that were zero. 
    
According to different facts from the services, reports, and articles, the maximum speed that a pedal assisted e-bike can achieve is 18-mph. A conservative value of 20-mph was used to define a speed outlier. Trips that had speeds over 20 mph were adjusted. The speed was set to 20 mph and the duration was adjusted since the distance isn’t variable.

$$\frac{Distance}{\frac{Duration}{60}} = 20 \longrightarrow \text{Solving for Duration} \longrightarrow Duration = 3 \times Distance$$

Round trips have a speed of 0-mph because they start and end at the same station which means their distance is 0 resulting in a speed of 0. For round trips the speed was set to the average speed of 6-mph. Using the same initial formula as before, the speed was set to a constant value and since duration is known, the distance column was updated. 

$$\frac{Distance}{\frac{Duration}{60}} = 6 \longrightarrow \text{Solving for Distance} \longrightarrow Distance = \frac{Duration}{10}$$

With the speed outliers handled the next cleaning task was to remove trips where the start time was after the end time. These trips were removed because the quantity of trips that had this error did not justify the cost associated with fixing them.

<p style="text-align:right"> <b> 1.2.B Trip Table Cleaning - Duration Outliers</b> </p>

The first issue when trying to clean the duration outliers is handling the million of rows that are in each of the five trip tables. Trying to determine outliers by querying the entire table is extremely inefficient and thus required a sample. However, using the built-in Bernoulli sampling of PostgreSQL was just as inefficient as querying the entire table. The samples not only had to be random, the number of trips selected in the sample had to be large enough, and the query had to be decently fast. To meet all three conditions a sampling procedure was created to get a million rows from each trip table. 

<b>The Sampling Procedure</b> <br>
<ol>
    <li> Use the  <code>TABLESAMPLE SYSTEM(1)</code> clause to random sample 1% of data.
    <li> Order the results randomly using <code>RANDOM()</code> because the system sample samples pages and not individual rows.
    <li> Grab the first 20,000 rows using <code>LIMIT 20000</code>.
    <li> Append the results of the query to the dataframe.
    <li> Repeat steps 1-4 until the number of samples is greater than or equal to the desired amount (1M samples = 50 repeats).
</ol>

<div style="clear: right;">
   <p style="float: right;"> 
       <figure style="float:right;text-align:center">
           <div>
               <img src="./Data/Images/Report/0009.Duration-Distribution.png" width="550" height="550" />
           </div>
           <div>
               <img src="./Data/Images/Report/0009.Duration-Distribution-2.jpg" width="550" height="550" />
           </div>
           <figcaption> Figure 3. Visualization of the Sample Duration Distributions </figcaption>
       </figure>
   </p>
   <p>From the boxplot it looks like all of duration distributions are similar in the sense that they are drastically skewed to the right. Standard deviation ranges from 2.9 – 19.2 hours and the 75th percentile barely breaks 20m; whereas the maximum values are in the thousands of minutes. Because of this skew, the mean and standard deviation were not reliable measures to use. With such a drastic skew, the quantiles are a significantly better measure to determine outliers as they are more representative of real life. Each different sample has their own distributions so there isn’t a one size fits all quantile measure that can be used for all the samples. 66 mins might be the 99th percentile for Service A, but the 99th percentile for Service B may be at 160 mins. The question then becomes: Do riders take the same length of trips, regardless of the service? If so, is there a universal duration time that could be used across all bike share services. 
    </p>
</div>

When asking if riders take the same length of trips, what is really being asked is if the duration of the samples taken are all pulled from the same underlying distribution. Comparing a single dependent variable across five different non-normal samples meet the conditions for the Kruskal-Wallis Test with the:

<ul>
    <li> Null Hypothesis $H_0$: The samples all originate from the same distribution and have the same median values.
    <li> Alternative Hypothesis $H_1$: At least one sample originates from a different distribution and has a different median. 
</ul>

The Scipy package was used to conduct this test and the p-value returned was 0; meaning there is enough evidence to reject $H_0$ at any significance level. The alternative hypothesis of the Kruskal-Wallis Test states that at least one sample originates from a different distribution, but it doesn’t say anything about how the samples compare pairwise.  On a pairwise level the Mann-Whitney U Test was used to compare the samples with the same $H_0$ and $H_1$ as the Kruskal-Wallis Test, but for pairs instead of a group. All 10 tests resulted in a p-value of 0, meaning that riders of different bike share companies don’t ride for the same amounts of time.

Although the statistical tests failed, it was important to try and find a single outlier value that could work for all the services. So instead a “Quantile Threshold” was looked for. The aim was to keep no less than 95% of all the trip data for each service. The search was for a pair of values, one for the lower end one for the upper, that would return <b> at least 95% </b> of the data for all services. To find those values the 97.5th quantile for every service was found and the highest of those values was taken as the upper cutoff. Similarly, the 2.5th quantile was found for every service and the lowest of the group was taken for the lower cutoff. For every service, trips with a duration above 87.3 minutes or below 2.21 minutes was removed from the trip table. Using those values the percentage of data kept ranged from 96 to 97.5%.

## II. Exploratory Data Analytics

Since the focus of the project is not on trip data, only basic analysis was done on the trip data directly. The trip data was mainly used as a secondary source to highlight analysis on the station data.

<a id="Back-To-Basic-EDA-Trips"> </a>

<p style="text-align:center;"> <b> 2.1 Basic Exploratory Analysis for Trips</b> </p>

The first question asked was how may trips were taken per month for each service across the service’s entire lifetime. 
The number of trips taken over time for each service, except Bay Wheels, ebbs and flows. This rise and fall may be due to the changing seasons from winter to summer and back in the four cities that the services are based in (Boston, Washington D.C. New York City, Chicago). Temperatures begin to rise into spring and reaches its heights in the summer and the number of trips taken does so as well. When temperatures drop going into fall/winter the number of trips drop as well. When it comes to Bay Wheels, this weather pattern doesn’t really apply because the winter climate in the San Francisco area is mild and rainy, more comparable to the spring climate in the other cities. 

<div style="clear: right;">
   <p style="float: right;"> 
       <figure style="float:right;text-align:center">
           <div>
               <img src="./Data/Images/Report/0010.Bay-Month.jpg" width="650" height="650" />
           </div>
           <div>
               <img src="./Data/Images/Report/0010.Blue-Month.jpg" width="650" height="650" />
           </div>
           <div>
               <img src="./Data/Images/Report/0010.Citi-Month.jpg" width="650" height="650" />
           </div>
           <figcaption> Figure 4. Number of trips per month for Bay Wheels, Blue Bike, & Citi Bike </figcaption>
           <p style="text-align:right"> <a href="#All-Months"> Link to A3. Trips Per Month - All Services </a> </p>
       </figure>
   </p>
</div>

In line with the NACTO report, over time, both peaks and troughs for the four services that have the seasonality pattern were gradually getting higher. Higher highs and higher lows implies an uptrend and in this context an uptrend is equivalent to increased ridership. 

As with all things, COVID-19 did influence the number of trips in 2020. In the first months of 2020 the number of trips had a massive drop off going into April instead of following its usual behavior of a steady rise.  When there are lockdowns in place, people aren’t leaving their house, let alone getting on a public bicycle. 

Drilling down a little further the number of trips was aggregated on a per hour basis. What stands out is that all of the graphs look the same for every service. For every hour across the country (taking into consideration timezones) the services receive the same pattern of traffic. 
<ol>
    <li> During the 06:00 hour the trips begin to increase hitting its peak during the 08:00 hour.
    <li> Then from 09:00 – 15:00 the number of trips remains stable.
    <li> A secondary volume increase begins at 16:00 reaching the daily high during the 17:00 and 18:00 hours.
    <li> Finally, the trips slowly taper off and repeats the cycle in the morning. 
</ol>   

It is likely that people are using the bike services to commute to work. This would explain the two demand spikes during the day as they coincide with rush hour times <a href="https://en.wikipedia.org/wiki/Rush_hour"> [4]</a>.

Naturally, the next question is to determine whether the cycle repeats every day of the week or do certain days differ from others. If the rush hour theory is true, then the weekend graph should behave differently than the weekday graph. It turns out that on an hour basis, weekdays have a similar structure different than that of the weekends; and the cycle above only applies to the weekdays. Weekends do not have the sharp rises and declines that are associated with rush hour. The number of trips gradually rises and then gradually falls; slightly resembling the path of the sun in the sky. 

<figure style="text-align:center">
    <img src="./Data/Images/Report/0011.Weekday-Structure.jpg" style="float: left; width: 45%; margin-right: 3%; margin-bottom: 0.5em;" />
    <img src="./Data/Images/Report/0011.Weekend-Structure.jpg" style="float: left; width: 45%; margin-right: 3%; margin-bottom: 0.5em;" />
    <p style="clear:both;">
    <figcaption> Figure 5. The general pattern of trips taken per hour on weekdays and weekends </figcaption>
</figure>

<a id="Station-Growth"></a>
<p style="text-align:center;"> <b> 2.2 Station Based Exploratory Analysis</b> </p>

Due to the dynamic nature of the number of stations in an ecosystem, the birth and death columns were added to the tables in the station schema. Using those birth certificate columns, the number of stations added and removed in any given year can be audited. Citi Bike is the clear winner when it comes to growth metrics: leading in most stations at launch, most stations overall, and most stations added in a single year.  

<figure style="text-align:center">
    <figcaption style="text-align:center;margin: 10px 0;"> Table 7. The total number of stations added within any year of a bike sharing service as well as the running total of active stations. </figcaption>
    <img src="./Data/Images/Report/0011.Station-Growth.jpg" style="max-width:65%" />
</figure>

<p style="text-align:right;font-size:12px"> <a href="#Pop-Contest"> Back to Popularity Contest </a> </p>

<p style="text-align:right"> <b> 2.2.A Popularity Contest</b> </p>


<div style="clear: right;">
   <p style="float: right;"> 
       <figure style="float:right;text-align:center">
           <figcaption style="text-align:center;margin: 10px 0;"> Table 8. The most popular station of a bike sharing service across different years. </figcaption>
           <img src="./Data/Images/Report/0011.Top-Stations.jpg" width="300" height="300" />
       </figure>
   </p>
   <p>Diving deeper into the station analysis the most popular stations for each service by year was found. Every trip a station was invovled in it got a point, regardless of whether it was a start or end station (<i> 2 points for a round trip </i>). 

BayWheels, CapitalBike, and DivvyBike introduced the idea of dockless bikes in 2018-06, 2020-06, and 2020-07 respectively. A dockless bike doesn't need to be retrieved or returned from a designated station and can be unlocked/locked at any public bike rack. When riders are given that option, they take advantage of it. In 2020, dockless rides were the most popular 'station' for all three service. This doesn't prove that riders use the dockless feature more than stations, in fact that isn't true. It only proves that the dockless bike option is something that is desirable to riders and would be used if a bikeshare service started implementing it into their ecosystem.
    </p>
<p style="text-align:center;font-style:italic"> There are infinitely many dockless stations and they are all used to come up with the total for the category, meaning it is an aggregate. Therefore comparing that aggregate column to a single station is unfair. If the totals were summed for all stations, comparing an aggregate to an aggregate, then rides to/from a designated station would win by a landslide. </p>

   <p>
As the number one ranked station in a service changes over time it is important to understand whether or not a newly created station is taking that spot or an old station is gaining popularity. When a reigning champion is displaced, the challenger was looked at to determine if it was a recently created or an older station. 

<b>Blue Bike</b> <br>
South Station was born 4 months after the MIT and Central Square station and still took the number 1 spot for 2015. However, MIT at Mass Ave. stayed the champion until 2020. So what happened? The MIT station is in the vicinity of the college and when the college slowed down due to COVID-19, the guess is that traffic dried up with it. However, Central Square is only a couple of block away in the same zip code. The 02139 could be a high traffic zip code overall and feature that could be important to the model prediction is if the zip code is within a certain distance of a major college (or any college). 

<b>Capital Bike </b> <br>
    Both reigning champions were born around the same time, but for some reason in 2014, Columbus Circle gained and maintained popularity. In fact, many stations over took Massachusetts Ave and over the years it slowly moved out of the top 5.
    
<b> Citi Bike </b> <br>
    All the top stations in Citi Bike were born at the same time and only 8 Ave was discontinued. When it comes to 2020, West St. might only be ranked number one, once again, due to COVID-19. The reason for this is because Pershing Square is across the street from Grand Central Terminal and three blocks away from Times Square, which are both ultra high traffic areas, except when the city is shutdown. 

 </p>
</div>

<a id="Pop-Contest"></a>
<p style="text-align:right"> <b> 2.2.B Popularity Contest USA</b> </p>

The top top stations within a service were found, but what about the top stations across all services. To compare stations across all services the number of trips a station recieved has to be relative to all the trips taken within the year. Otherwise, Citi Bike stations would win all the time. 

<figure style="text-align:center">
    <figcaption style="text-align:center;margin: 10px 0;"> Table 9. The most popular stations across all bike sharing services. </figcaption>
    <img src="./Data/Images/Report/0011.Top-10.jpg" style="max-width:70%" />
</figure>

The percentage of trips taken using a dockless bike in Bay Wheels hit an astronomical value of 86% in 2020. The belief is that COVID-19 played a role in the <b> overall </b> reduction of trips in gneral, but confidence is low attributing the same reasoning to the massive spike in dockless bike use.

One thing that can be thought of combines the the ability of dockless bikes being parked anywhere with the state lock down. People that WERE using the bikeshare were going directly to where they needed to go, parked the bike, picked up the same bike, then parked it very close to their apartment, and repeated the process with the same bike for another trip. 

<p style="text-align:center"> <i>This explanation is really grasping at straws and there isn't any data to back it up. This could be explored using the bikeID data, which was removed when planning the design of the database.</i> </p>

Other than dockless stations, "old-school" CapitalBike stations (2010-2012) appeared 5 times in the top 10 stations. Why? Looking back at the <a href="#Station-Growth"> station growth table</a>, in those years CapitalBike had 106, 144, and 194 stations in its ecosystem, respectively. Those numbers represent some of the lowest numbers of stations in an ecosystem at any given time. Which such a low number of stations, stations that are more popular will be able to capture more of the rider market because the system isn't "decentralized". Which is why Citi Bike makes it appearances in the 80th - 110th (out of 111) places. From its inception, Citi Bike had the most stations out of every ecosystem and no other service could keep up with its growth. CitiBike has nearly double the number of stations as its closet competitor. You can say that the CitiBike ecosytem is "decentralized" and the number of trips is distributed "evenly" across the ecosytem.

<p style="text-align:right"> <b> 2.2.C Inter Zip Code Travel</b> </p>

The significance of this project is to help a bike share company expand into a new area. For an expansion into a new area, how important is it to expand into multiple zip codes? If it is importnant, how many zip codes should the expansion cover? To determine this the number of trips that started and ended in different zip codes was compared to the total number of trips.

<figure style="text-align:center">
    <img src="./Data/Images/Report/0012.Interzip-Travel.jpg" style="max-width:50%" />
    <figcaption> Figure 6. Boxplot of the number of trips that start and end in different zip codes. </figcaption>
</figure>

The boxplot reveals that it is extremely important for stations to be set up in multiple zip codes when planning an expansion effort. The majority of trips taken by riders, in all services, start in one zipcode and end in another. It's safe to say that not having a multiple zip code expansion wouldn't be desirable to riders. To get an idea of how things should be done, the number of zip codes the stations were spread across when the five services made their inital launches was determined. Additionally, the number of zip codes the stations were in at the end of 2020 was determined. 

<figure style="text-align:center">
    <figcaption style="text-align:center;margin: 10px 0;"> Table 10. The initial number of zip codes a bike sharing service launched in and the current number of zip codes it covers. </figcaption>
    <img src="./Data/Images/Report/0013.Zip-Expand.jpg" style="max-width:50%" />
</figure>

<p style="text-align:right"> <b> 2.2.D How Many People Does Each Station Serve?</b> </p>

When people use public transportation they go to the spot that is most convient for them. Typically, conveient means the closet. I say typically because there are times when people have to go farther distances to catch a bus or train because the route of the bus/train is more conveient for where they are headed. However, in the case of bike share, there is no incentive to go to a bike station that is farther away from the one that is closet to you. 

With that being said, a station only serves the people that are closer to it than to any other station. The number of people a station serves is defined by the equation $$S(s) = \sum_{i=1}^{N}\frac{A(G_i \cap V_s)}{A(G_i)} \cdot P_i$$ where $S(s)$ is the number of people served by station $s$, $N$ is the number of neighborhoods in the region (NYC or San Francisco), $A$ is the area function, $G_i$ is the geometry polygon for neighborhood $i$, $V_s$ is the voronoi polygon for station $s$, $P_i$ is the population for neighborhood $i$. 

<p style="text-align:center;font-style:italic"> <b>A Simple Interpretation of the Formula: It's multiplying the portion of a station's voronoi polygon that is in a neighborhood by the population density of that neighborhood.</b></p>

For every station, every neighborhood in the region needs to be iterated through completing the list of steps below. To complete this analysis the population data from the nyc_profile and sanfran_profile in the neighborhoods schema, a series of python functions, and built in PostGIS functions such as <code>VoronoiPolygons</code> were used.
<ol>
    <li> Intersect the neighborhood geometry with the voronoi geometry of the station.
    <li> Find the area of that intersection.
    <li> Divide that area by the area of the neighborhood geometry.
    <li> Muliply that number by the population of the neighborhood.
    <li> Save the number.
    <li> Repeat the previous steps for all the neighborhoods.
    <li> Add up all the saved numbers.
</ol>

<figure style="text-align:center">
    <img src="./Data/Images/Report/0014.NYC-Serve-Box.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <img src="./Data/Images/Report/0014.NYC-Serve-Kernel.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <img src="./Data/Images/Report/0014.NYC-Serve-Strip.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <p style="clear:both;">
    <figcaption> Figure 7. Different visulizations of the number of people served by bike stations in New York City. </figcaption>
</figure>
Each of the three graphs above show the number of riders served by stations in NYC. They reveal that majority of stations, about 75%, serve between 100K and 225K people. There is a smaller group that serve between 225K and 350K people. NYC can be broken up into 5 boroughs (only 4 have stations), and the graph below shows the number of riders served on a borough basis. 

<figure style="text-align:center">
    <img src="./Data/Images/Report/0015.NYC-Serve-Borough.jpg" style="max-width:50%" />
    <figcaption> Figure 8. Strip plot of the number of people served by bike stations in NYC by borough </figcaption>
</figure>

For the San Francisco stations the same process can be repeated. In San Francisco the majority of stations serve between 10K and 60K people, again about 75% of the stations. The people served graphs are shown below. 

<figure style="text-align:center">
    <img src="./Data/Images/Report/0016.SF-Serve-Box.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <img src="./Data/Images/Report/0016.SF-Serve-Kernel.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <img src="./Data/Images/Report/0016.SF-Serve-Strip.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <p style="clear:both;">
    <figcaption> Figure 9. Different visulizations of the number of people served by bike stations in San Francisco. </figcaption>
</figure>

Although the people served statistic is interesting, it isn't very useful on in its own. It's impossible to tell if a station with a higher statistic has a bigger voronoi area or has a smaller voronoi area in a denser part of the city. A better statistic to look at would be the ratio between the riders served and the area of the voronoi. The graphs of the ratios for both cities are shown in the graphs below:

<figure style="text-align:center">
    <img src="./Data/Images/Report/0017.NYC-Serve-Ratio-Box.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <img src="./Data/Images/Report/0017.NYC-Serve-Ratio-Kernel.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <img src="./Data/Images/Report/0017.NYC-Serve-Ratio-Borough.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <p style="clear:both;">
    <figcaption> Figure 10. Different visualizations of the <b> ratio </b> between people served and the voronoi area of stations in New York City</figcaption>
</figure>

<figure style="text-align:center">
    <img src="./Data/Images/Report/0018.SF-Serve-Ratio-Box.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <img src="./Data/Images/Report/0018.SF-Serve-Ratio-Kernel.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <img src="./Data/Images/Report/0018.SF-Serve-Ratio-Strip.jpg" style="float: left; width: 30%; margin-right: 3%; margin-bottom: 0.5em;" />
    <p style="clear:both;">
    <figcaption> Figure 11. Different visualizations of the <b> ratio </b> between people served and the voronoi area of stations in San Francisco</figcaption>
</figure>

Looking at just riders served the data was really spread out, the data is much tighter when looking at the ratio between the riders served and the area of the voronoi. Regardless of the borough, regardless of the location, the number of people that a station serves in NYC is rarely over 3.5 people per square meter of it's voronoi polygon. Which makes practical sense because the denser the population of an area the more stations you need to accomodate the population. The more stations packed into one area, the smaller the voronoi area. Although the area is small, it is still serving tons of people. Population density may be an extremely important factor when a company chooses the number of; and the locations of stations in a potential expansion area. 

Looking at the ratio between the riders and the area of the voronoi for San Francisco, the number of people that a station serves is rarely over 0.5 people per square meter of it's voronoi. New York City is bigger and more dense that San Francisco, so it makes sense that the ratios of people served per square meter are larger in NYC. 

<p style="text-align:right"> <b> 2.2.E Would You Have Bike Access?</b> </p>

With an understanding of how many people each station serves the next question to ask is: would you even be served to begin with? What is the probability that a randomly selected New Yorker or San Franciscan lives within a certain distance of a station? This question is asking about the <b>accessibility</b> of the overall network of stations. How many people in the city have the OPTION of using bike sharing if they wanted it.

The methodology is to pick a random person from the city, represented as a point. A buffer will then be created around the point; used as a search radius to locate any stations within it. For New York City this search process is done with Citi Bike stations AND with MTA Subway Entrances to see how they comapre in terms of accessibility. For San Francisco only the BayWheels stations are used. Additionally, the accessibility is determined using all neighborhoods in a region as well as only the subset of neighborhoods that have stations. 

Each experiment picked 100 random people and determined what percentage of them lived within a certain distance from a station. For each distance, 100 rounds of the experiment were completed and the average percentage was taken as the final metric. Most people are willing to walk for five to ten minutes, or approximately a quarter/half a mile to a transit stop <a href="https://safety.fhwa.dot.gov/ped_bike/ped_transit/ped_transguide/ch4.cfm#:~:text=A.-,Typical%20Walking%20Distance%20to%20Transit,when%20accessing%20heavy%20rail%20services"> [5]</a>. Therefore the distances that were chosen where:
<ul>
    <li> 0.5 (805m) - A half a mile, approximately a 10-minute walk
    <li> 0.375 (604m) - A three-quarter mile, approximately a 7.5 minute walk
    <li> 0.25 (402m) - A quarter of a mile, approximately a 5-minute walk
    <li> 0.125 (201m) - An eight of a mile, aproximately a 2.5-minute walk
    <li> 0.05 (80m) - Approximately a 1-minute walk
</ul>

<figure style="text-align:center">
    <figcaption style="text-align:center;margin: 10px 0;"> Table 11. The percentages of people that live within a certain distance of a bike sharing station. </figcaption>
    <img src="./Data/Images/Report/0019.Access.jpg" style="max-width:80%" />
</figure>

<p style="text-align:center"> <b>Observations</b> </p>
<ol>
    <li style="margin:10px 0">When it comes to the subset of neighborhoods that have stations, Citi Bike outperforms the subway in terms of accessibility. On the flip side, in terms of all neighborhoods, the subway does better. If MTA bus stations were included, I believe that the MTA as a whole would completely dominate Citi Bike in all categories and distances.
    <li style="margin:10px 0"> When all neighborhoods are included, the percent of people that have access to CitiBike is cut in half for all distances. Whereas the drop for the subway stations isn't as sharp. This shows that the distribution of stations is concentrated in a 'small' number of neighborhoods. When the no-station neighborhoods are added back into the 'neighborhood set' there were so many of them (neighborhoods) that they caused the probability to drop significantly.
    <li style="margin:10px 0"> The BayWheels stations of San Francisco are more spread throughout the neighborhoods than the Citi Bike stations of NYC. The drop in probability between the two sets of neighborhoods is less sharp (they are only about 10% away from each other).
    <li> For the neighborhoods that have stations, at the lower distances Citi Bike is more accessible. I believe that this is because Citi Bike concentrates more stations to neighborhoods with higher densities. The more dense a neighborhood, the more stations, and the more likely a person will be randomly picked from those neighborhoods.
    <li> It's very rare to live within a 1 minute walk from any transit service. 
</ol>

## III. Zip Code Station Predictions
To predict the number of stations that should be in a given zip code, a count of the known number of active stations that exist in the handful of zip codes that are in the fve sharing services are taken. Using the zip code features from the zipcodes_profile table in the database, this count will be the target value in our supervised learning regression problem. After the model is trained a pipeline will be built to take in the information that for all the zipcodes that don't already have bike stations.   

<p style="text-align:center;"> <b> 3.1 Zip Code Data Cleaning</b> </p>

The zip code data was added to the database as is. Unlike the trip data, the zip code data is small enough to be manipulated locally. More importantly, having raw data in the database preserves the option of cleaning it how we see fit when the time comes as compared to cleaning it and then "locking in" that cleaning. 

<p style="text-align:right"> <b> 3.1.A Initial Data Import</b> </p>

The aim of the project is to come up with a universal regression model that can be used for any of the zip codes in the states listed. With that in mind the model shouldn't have an understanding of states. For modeling purposes the state names and abbreviations were removed from the data and saved in a separate dataframe. For prediction purposes the zip codes were first filtered by state names and then those columns were removed. 

<p style="text-align:right"> <b> 3.1.B Handling Missing Values - Basic Fills & Deletes</b> </p>


<div style="clear: right;">
   <p style="float: right;"> 
       <figure style="float:right;text-align:center">
           <figcaption style="text-align:center;margin: 10px 0;"> Table 12. Columns w/ missing values. </figcaption>
           <img src="./Data/Images/Report/0020.Missing-Values.jpg" width="300" height="300" />
       </figure>
   </p>
   <p>The first column of missing values handled was the Core-Based Statistical Area (CBSA) column. A CBSA is one or more adjacent counties or county equivalents that have at least one ubran core area of at least 10,000 population, plus adjacent territory that has a high degree of social and economic integration with the core as measured by community ties <a href="https://www.census.gov/programs-surveys/metro-micro/about.html#:~:text=Each%20metropolitan%20statistical%20area%20must,but%20less%20than%2050%2C000%20population"> [6]</a> <a href="https://en.wikipedia.org/wiki/Core-based_statistical_area">[7]</a>. There are 927 CBSAs in the United States, each containing multiple zipcodes, categorized into two types: 

<ol>
    <li> Metropolitan: one urbanized area of 50,000 or more inhabitants.
    <li> Micropolitan: one urban cluster of at least 10,000 but less than 50,000 population.
</ol>

To fill the missing CBSA values, two files were combined to get as many missing values as possible. The Delineation file and the USPS Crosswalk file, when combined contains the CBSA code, all of the zip codes within that code, and which of the two categories of CBSA it falls into. The combined dataset was used to fill in any missing value. Any zip code still missing a CBSA isn't in a CBSA and the value in that column was set to the string 'None' so that the computer doesn't recognize it as a missing value when trying to train the model. 

In the table of missing values to the right there are a lot of columns that have a missing value count of 2770. This coulnd't be a coincidence and lead me to believe that there is a set of rows where all of those columns are blank. The idea turned out to be true and those rows were outright deleted since there was no viable solution to filling them. 
    
Deleting those 2770 rows helped with other columns, but there were still missing values to be handled. The next column of missing values to fill were the median age. Instead of filling the missing values with the median age of the United States as a whole (40.9), 607 random numbers were pulled from a normal distribution with a mean and standard deviation the same as the median age distribution. Although the data doesn't come from a normal distribution, the goal was to try to conserve the underlying distribution as much as possible.  
    </p>
</div>

<div style="clear: left;">
   <p style="float: left;"> 
       <figure style="float:left;text-align:center">
           <div>
               <img src="./Data/Images/Report/0021.KDE-Plots.jpg" width="400" height="400" />
           </div>
           <div>
               <img src="./Data/Images/Report/0022.Household-Size.jpg" width="400" height="400" />
           </div>
           <figcaption> Figure 12. <i> Top: </i> The KDE plots of the median age using different fillna methods.</figcaption>
           <figcaption> Figure 13. <i> Bottom: </i> The distribution of the average household size.</figcaption>
       </figure>
   </p>
</div>

Next was the household size. The distribution of average household size is outrageously skewed. The 75th percentile is 2.75 people per household and the maximum is 26 people. To fill missing values for the average household, the average household size for 2020 (2.53) was used.

The percent of the population in college or graduate school column is calculated using the population in college or graduate shcool and the total population. In the dataset if the number of people in college is 0, then the percentage column is NaN. So those values were filled with zeros.

The zipcode_segment describes the density and the type of people that are working in the area. This segmentation could be really useful, but it is missing for 93% of the data so it was dropped.  

<p style="text-align:right;color:#669BBC"> <b> 3.1.C Handling Missing Values - Machine Learning Imputation</b> </p>

The final four columns with missing data are listed below. These columns are columns where the missing data could be determined using the other columns via machine learning techniques. Additionally, there is more than enough non-missing data in the dataset that can be used to train a model to determine the missing data.

<ol>
    <li> Median Household Income
    <li> <strike> Family Income Per Capita </strike>
    <li> <strike> Median Price for Owner Occupied Housing </strike>
    <li> Median Rent for Renter Occupied Housing
</ol>

Luckily, as this isn't the main machine learning portion of the section, there are two methods that streamline missing data imputation: Datawig's Simple Imputer and the MICE algorithm. Datawig looks like the cooler method, but the draw back to Datawig is that it can only impute one column at a time. Why is this an issue? Let's look at the correlation plot below. 


<figure style="text-align:center">
    <img src="./Data/Images/Report/0021.Original-Model-Correlation.png" style="max-width:85%" />
    <figcaption> Figure 14. Correlation heatmap of the features in the feature space </figcaption>
</figure>

The columns that are missing are correlated with each other, for example median household income and median rent have a correlation coefficient of 0.73. If datawig were to be used there would only be two unideal choices:
<ol>
    <li> Use all the columns except the four mentioned to predict the four missing columns individually in four seperate runs. The problem with this is that the correlations would be excluded which would be valuable to the predictions. 
    <li> Use all the data to predict the missing values for a single column. The problem with this is that the predictions would differ depending on which missing column was chosen to impute first. 
</ol>


<p style="text-align:center" > <b>The correlation plot exposes some multicollinearity that occurs via redundant columns so they need to be dropped before continuing to the MICE Algorithm. <i> Note: two of the four columns planned to be filled filled with MICE were dropped </i>
</b> </p>

<input type="checkbox" checked="True"> Total population and Total Population over 18 are the basically the same columns with the same correlation coefficients. Total Population over 18 is kept because Bike Share companies have a minimum rider age of 16. </input>

<input type="checkbox" checked="True">  Percent labor force employeed over 16, Family Income Per Capita and Median Household Income are behaving the same way as the total population pair. Median Household income is conserved because it is the most correlated with the variable that is being predicted (<i>Not Shown</i>). </input>

<input type="checkbox" checked="True">  Again, Median Price of Owner Occupied Units and Median Rent of Renter Occupied Units behave the same way the total population pair behaved. They are equally correlated with the variable that is being predicted (<i>Not Shown</i>). Homeowners typically also own cars, so the owner occupied units is removed. </input>

<input type="checkbox" checked="True">  Although not seemingly related like the other columns, Resident Last Year and Population in College are also correlated with the same columns in the same way. College population is kept because as was shown in the popularity contest in BlueBike the college population could be very important.</input>

<input type="checkbox" checked="True">  For an expansion it is highly desirably to expand into a location where people are currently living. Occupied Housing is kept over Total Housing.</input>

<input type="checkbox" checked="True">  Of all the Percent Units columns that are 2 units and greater, the 5-9 is kept because it is most correlated with the variable being predicted (<i>Not Shown</i>) </input>

<figure style="text-align:center">
    <img src="./Data/Images/Report/0021.Reduced-Model-Correlation.png" style="max-width:85%" />
    <figcaption> Figure 15. Correlation heatmap of the features in the feature space after removing multicollinearity </figcaption>
</figure>

The MICE algorithm only takes in numerical values and the ziptype and CBSA categorical columns need to be encoded. In the ziptype column there are two main types of zip codes <a href="https://udsmapper.org/zip-code-to-zcta-crosswalk/"> [8]</a>:
<ol>
    <li> Zip Code Areas: typical ZIP Codes without any special designation. 
    <li> Post Office or large volume customer: typically occupy a very small geography and have few residential addresses located within them. 
</ol>

The ziptype was ordinally encoded because there is an implicit ordering between the two types. A post office zip code doesn't have many residental addresses located within them and low residental addresses = low people = low bike stations. 

<p style='text-align:center'> <i> Note: There was a third ziptye called 'populated ZCTA, missing zip' but it only had 16 entries so those zip codes were deleted </i> </p>

For the CBSA column, a simple one-hot encoding was used. With the categorical columns taken care of the MICE algorithmn was applied using Scikit's IterativeImputer.

<p style="text-align:center;"> <b> 3.2 The Supervised Machine Learning - Making the Predictions</b> </p>

Only the subset of zipcodes that have bike share stations are used to build a model. This machine learning task can be approached as a regression problem as well as a classification problem. Regression is straight forward, the aim would be to predict the exact number of stations that should be in a zip code. On the flip side, the number of stations could be binned into segments and then the segment could be predicted. Regression outputs an exact recommendation, whereas classification outputs a range as a class. This project is approaching the problem from a regression approach. 

<p style="text-align:right"> <b> 3.2.A Zip Codes Clustering I - All Zip Codes</b> </p>

To begin, the mean shift unsupervised learning algorithm was used to cluster all the zip codes in the United States. The results of the clustering became a new feature that was used to train the model. The clustering produced 156 clusters with a silhouette score of 0.57 and clustered the majority of zip codes (38560) into a single cluster.

With the new feature in place, the number of stations for each zip code was queried from the database and merged into the dataframe containing all the zip codes. As mentioned before, the data was then partitioned, taken only the zip codes that have stations within them. <b> It is this partition that the model was trained on, the no-station zip codes will get feed into the pipeline after the model is created. </b>

<figure style="text-align:center">
    <img src="./Data/Images/Report/0022.Zipcodes-Stations.jpg" style="max-width:65%" />
    <figcaption> Figure 16. Boxplot of the number of stations in zip codes across all the services </figcaption>
</figure>

<p style="text-align:right"> <b> 3.2.B Zip Codes Clustering II - "Has Station" Zip Codes </b> </p>

Another mean shift clustering was performed only on the subset of zip codes that have stations. Again, the outputs of the clustering will be used as another column that can be used to predict the number of stations. <i> However, this time, the column won't be built into the no-station zipcodes dataset and the meanshift model (along with the Robust Scaler) used to do the clustering will be part of the pipeline created to predict the number of stations. <i>

<a id="Regression"> </a>

<p style="text-align:right"> <b> 3.2.C Regression Models </b> </p>

After spliting the data using a 70-30 split, both the features and labels of the training set were scaled separately using seperate RobustScalers. The scaling was then used to transform the testing set. Additionally the scalers used were saved to be used in the pipeline for making the predictions. 

Bikeshare stations aren't cheap and there is a lot of time, money, and energy being invested into an expansion effort. Even with Robust Scaling the number of stations is spread out and the models tend to have a high maximum error, which effects the RMSE significantly. Because of this, the Median Absolute Error was the primary measure used for for model tuning and model evaluation. The RMSE was looked at as a secondary measure in the evaluation. 
    
Five models were used in the training and the results of each model can be seen in the table below. Leveraging the AWS cloud via Sagemaker, the instance type was able to be scaled to hypertune all the models. The tuning was done using TuneSearchCV with hyperopt search optimization and RepeatedKFolding using the default settings (<i>10 repeats of 5 folding</i>). 

<figure style="text-align:center">
    <figcaption style="margin: 10px 0;"> Table 13. The training/testing results for the model candidates used in the project </figcaption>
    <img src="./Data/Images/Report/0023.Model-Results.jpg" style="max-width:90%" />
    <p style="text-align:right"> <a href="#Model-Errors"> Link to A4. Model Plots - Distribution of Errors </a> </p>

</figure>


The Ridge Regression model is chosen as the final model used to make the decision, but the support vector model was also used to make predictions. Two identical pipeline were built minus the final step which used only one of the two models as the predictor. The data that gets fed into the pipeline is the original zipcode data with the "all-zip code" mean shift clustering (3.2.A). The pipeline steps are shown in the figure below:
    
<figure style="text-align:center">
    <img src="./Data/Images/Report/0023.Pipeline.png" style="max-width:100%" />
    <figcaption> Figure 17. A visual representation of the pipeline </figcaption>
</figure>
    
The models predicted very similar results. Both mainly predicting 0 stations for the majority of zip codes (7700+ out of 9492) and had maximum predictions of about 25. The high number of 0 predictions isn't very suprising being that all the bike sharing services are located in the largest cities of the country (by population). My guess is that the farther a zip code is away from the heart of the city the less stations they get and at some point they shouldn't get any. Bike Sharing isn't suited for areas where a long journey is the norm and a car is really the only appropriate form of transportation. Imagine riding a bike 10 miles to the supermarket (25m BY CAR) and there are no more bikes when you try to go home. Without the other transportation options that urban areas offer, you would have to walk that 10 miles, groceries in hand.
    
<figure style="text-align:center">
    <img src="./Data/Images/Report/0024.Final-Prediction.jpg" style="max-width:85%" />
    <figcaption> Figure 18. The number of stations predicted in different zip codes </figcaption>
</figure>

<p style="text-align:center"> <b> Dear CitiBike, even with all the zeros that the model predicted, 15 stations were recommended for the zip code where I live. Maybe one day you could do an expansion into my area. One spot I recommend is outside my house. </b> </p> 
    
<div style="line-height:11px">
    <p style="text-align:right;font-style:italic;color:#c1121f"> <b> Data Science = Solving Problems = Happiness </b> </p>
    <p style="text-align:right;"> <b> Denzel S. Williams <b> </p>
</div>

<hr>

### A1. Project Improvements & Extenstions

<b> Upgrades </b> <br>
In the future I would like to use an ETL tool to directly take the data from the different S3 buckets every month and have it stream into the database instead of doing an ETL job by hand. To complete this I'm thinking that I would have to create a script that downloads the data from the company's S3 bucket once a month when it is updated. I could possibly use an event to trigger an AWS Lambda function to do this. Or I may be able to bypass that with AWS Glue. 

<b> Improvements </b> <br>
One imporvement that could be done to the project is getting more data. There are more bike share companies in the US beyond the five major ones used in the project and some of them post their tripdata to S3 buckets as well. Having more bike share data provides me with more actual station data that can be used to improve the model. 

The most beneficial data improvement would be to get more zip code data. There are some really interesting zip code datasets that exist online, but they are pricey.

<ol>
    <li style="margin: 10px 0"> <a href="https://www.esri.com/content/dam/esrisites/sitecore-archive/Files/Pdfs/library/brochures/tapestry-segmentation.pdf"> ESRI Tapestry Segmentation:</a> Going beyond demographic data it breaks down zipcodes into lifemode and urbanization summary groups. For example, a zip code expansion would probably work best in "principal urban centers" which are categorized as "young, mobile, diverse in metros of 2.5+ million people"". 
    <li style="margin: 10px 0"> <a href="https://www.incomebyzipcode.com/"> Income by Zipcode: </a> This service gives the most current Census Bureau income statistics for United States Zip Codes.
    <li> <a href="https://www.melissa.com/v2/lookups/zipclimate/zipcode/"> Climate Data: </a> The EDA revealed that the number of trips ebbs and flows as the winter comes and goes. Which leads me to believe that the climate of a zipcode should play a role in the number of bike share stations. Bike share stations probably won't do well in year-round winter style climates. 
</ol>

<b> Extensions </b> <br>
One thing that I want to look at is to try to come up with not on the quantity, but the locations of bike share stations in a zip code. Going even further I could try to constrain the "people served per station ratio" to some number all while maximizing the odds of access. Drilling down EVEN further, I could also try to come up with the number of bikes that should be at a proposed station. 

Another cool extension would be to create a network graph of the stations in Citi Bike. Where stations are only connected to each other if the number of trips taken between them reach a certain threshold. For example if 1000 trips started and ended with the same pair of stations they are considered connected. 

<a id="Dataset-Usage"></a>

### A2. Dataset Usage
<p style="font-size:12px"> <a href="#Jump-To-Cleaning"> Back To Extract <a> </p>
<figure style="text-align:center">
    <figcaption> Table A1: How each dataset was used in the project and its database location </figcaption>
    <img src="./Data/Images/Report/A025.Datasets.jpg" style="max-width:85%" />
</figure>


<a id="All-Months"> </a>

### A3. Trips Per Month - All Services
<p style="font-size:12px"> <a href="#Back-To-Basic-EDA-Trips"> Back to Basic EDA for Trips </a> </p>

<figure style="text-align:center">
    <img src="./Data/Images/Report/A027.Bay-Month.jpg" style="max-width:80%" />
    <figcaption> Figure A1. The number of trips taken per month in Bay Wheels </figcaption>
</figure>

<figure style="text-align:center">
    <img src="./Data/Images/Report/A027.Blue-Month.jpg" style="max-width:80%" />
    <figcaption> Figure A2. The number of trips taken per month in Blue Bike </figcaption>
</figure>

<figure style="text-align:center">
    <img src="./Data/Images/Report/A027.Capital-Month.jpg" style="max-width:80%" />
    <figcaption> Figure A3. The number of trips taken per month in Capital Bike </figcaption>
</figure>

<figure style="text-align:center">
    <img src="./Data/Images/Report/A027.Citi-Month.jpg" style="max-width:80%" />
    <figcaption> Figure A4. The number of trips taken per month in Citi Bike </figcaption>
</figure>

<figure style="text-align:center">
    <img src="./Data/Images/Report/A027.Divvy-Month.jpg" style="max-width:80%" />
    <figcaption> Figure A5. The number of trips taken per month in Divvy Bike </figcaption>
</figure>

<a id="Model-Errors"> </a>

### A4. Model Plots - Distribution of Absolute Errors
<p style="font-size:12px"> <a href="#Regression"> Back to Regression Models </a> </p>

<figure style="text-align:center">
    <img src="./Data/Images/Report/A026.Linear-Model.jpg" style="max-width:50%" />
    <figcaption> Figure A6. Distribution of absolute errors made by the Linear Model </figcaption>
</figure>

<figure style="text-align:center">
    <img src="./Data/Images/Report/A026.Ridge-Model.jpg" style="max-width:50%" />
    <figcaption> Figure A7. Distribution of absolute errors made by the Ridge Model </figcaption>
</figure>

<figure style="text-align:center">
    <img src="./Data/Images/Report/A026.Support-Model.jpg" style="max-width:50%" />
    <figcaption> Figure A8. Distribution of absolute errors made by the Support Vector Model </figcaption>
</figure>

<figure style="text-align:center">
    <img src="./Data/Images/Report/A026.RF-Model.jpg" style="max-width:50%" />
    <figcaption> Figure A9. Distribution of absolute errors made by the Random Forest Model </figcaption>
</figure>

<figure style="text-align:center">
    <img src="./Data/Images/Report/A026.GB-Model.jpg" style="max-width:50%" />
    <figcaption> Figure A10. Distribution of absolute errors made by the Gradient Boosting Model </figcaption>
</figure>

### A5. References
<ol style="margin: 10px 0;">
    <li> “Micromobility.” Wikipedia, Wikimedia Foundation, 24 Mar. 2021, <a href="https://en.wikipedia.org/wiki/Micromobility"> en.wikipedia.org/wiki/Micromobility. </a> 
    <li> Ink, Social. “Shared Micromobility in the U.S.: 2019.” National Association of City Transportation Officials, <a href="https://nacto.org/shared-micromobility-2019/"> nacto.org/shared-micromobility-2019/. </a>
    <li> How Many Cities Are In The Us, <a href="https://worldpopulationreview.com/us-city-rankings/how-many-cities-are-in-the-us"> worldpopulationreview.com/us-city-rankings/how-many-cities-are-in-the-us. </a>
    <li> “Rush Hour.” Wikipedia, Wikimedia Foundation, 22 Jan. 2021, <a href="https://en.wikipedia.org/wiki/Rush_hour"> en.wikipedia.org/wiki/Rush_hour. </a>
    <li> “Pedestrians and Transit - Safety: Federal Highway Administration.” Safety, <a href="https://safety.fhwa.dot.gov/ped_bike/ped_transit/ped_transguide/ch4.cfm#:~:text=A.-,Typical%20Walking%20Distance%20to%20Transit,when%20accessing%20heavy%20rail%20services."> safety.fhwa.dot.gov/ped_bike/ped_transit/ped_transguide/ch4.cfm#:~:text=A.-,Typical%20Walking%20Distance%20to%20Transit,when%20accessing%20heavy%20rail%20services. <a>
    <li> Bureau, US Census. “About.” The United States Census Bureau, 1 Apr. 2020, <a href="https://www.census.gov/programs-surveys/metro-micro/about.html#:~:text=Each%20metropolitan%20statistical%20area%20must,but%20less%20than%2050%2C000%20population"> www.census.gov/programs-surveys/metro-micro/about.html#:~:text=Each%20metropolitan%20statistical%20area%20must,but%20less%20than%2050%2C000%20population. </a>
    <li> “Core-Based Statistical Area.” Wikipedia, Wikimedia Foundation, 2 Mar. 2021, <a href="https://en.wikipedia.org/wiki/Core-based_statistical_area"> en.wikipedia.org/wiki/Core-based_statistical_area. </a>
    <li> “ZIP Code to ZCTA Crosswalk.” UDS Mapper, 1 Jan. 1970, <a href="https://udsmapper.org/zip-code-to-zcta-crosswalk/"> udsmapper.org/zip-code-to-zcta-crosswalk/. </a>
</ol>